### *Assessment of pathways using the IAMC 1.5°C Scenario Data*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Economic growth per capita

This notebook computes the variable "GDP/capita" and plots a number of figures for all pathways and for pathways limiting global warming to below 1.5°C by the end of the century.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load ``pyam`` package and other dependences

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('../assessment/style_sr15.mplstyle')
%matplotlib inline
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators`
in the repository https://github.com/iiasa/ipcc_sr15_scenario_analysis.
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r2.0.xlsx')

In [ ]:
sr1p5.load_meta('../assessment/sr15_metadata_indicators.xlsx')

In [ ]:
with open("../assessment/sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
cats_15 = specs.pop('cats_15')
cats_15_no_lo = specs.pop('cats_15_no_lo')
cats_2 = specs.pop('cats_2')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
df = sr1p5.filter(category=cats, year=range(2010, 2101, 10))

## Compute 'GDP/capita' and append to `IamDataFrame`

In [ ]:
pop = (
    df.filter(variable='Population')
    .convert_unit('million', 'billion', factor=1/1000)
    .timeseries()
)
pop.index = pop.index.droplevel([2, 3, 4])

gdp = (
    df.filter(variable='GDP|PPP')
    .convert_unit('billion US$2010/yr', 'trillion US$2010/yr', factor=1/1000)
    .timeseries()
)
gdp.index = gdp.index.droplevel([2, 3, 4])

In [ ]:
gdp_pop = gdp / pop
gdp_pop['variable'] = 'GDP/capita'
gdp_pop['unit'] = 'billion US$2010/yr/cap'
gdp_pop['region'] = 'World'

Remove scenarios that don't report GDP

In [ ]:
gdp_pop = gdp_pop[gdp_pop[2020] > 1]

In [ ]:
df.data = df.data.append(pyam.IamDataFrame(gdp_pop).data)
gdp_pop = df.filter(variable='GDP/capita')

## Draw figures for all pathways

In [ ]:
gdp_pop.line_plot(color='category', fill_between=True, final_ranges=True)

In [ ]:
gdp_pop.line_plot(color='category', final_ranges=True)

In [ ]:
gdp_pop.line_plot(color='category', linewidth=0.5, final_ranges=True)

## Draw figures only for pathways limiting global warming to 1.5°C by the end of the century

In [ ]:
gdp_pop_15 = df.filter(variable='GDP/capita', category=cats_15)

In [ ]:
gdp_pop_15.line_plot(color='category', fill_between=True, final_ranges=True)

In [ ]:
gdp_pop_15.line_plot(color='category', final_ranges=True)

In [ ]:
gdp_pop_15.line_plot(color='category', linewidth=0.5, final_ranges=True)